<a href="https://colab.research.google.com/github/MichaelAshton/logistic-regression/blob/master/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

In [1]:
from google.colab import files
files.upload()

Saving us-census-data.zip to us-census-data.zip


{'us-census-data.zip': b'PK\x03\x04-\x00\x08\x00\x08\x00\xf4l\xeeJ\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\x0e\x008\x00adult-test.csv\x01\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\n\x00 \x00\x00\x00\x00\x00\x01\x00\x18\x006\xffC\xa5\xa6\xfc\xd2\x016\xffC\xa5\xa6\xfc\xd2\x016\xffC\xa5\xa6\xfc\xd2\x01\xbc]\xdbr$\xb7\x91}\xf7W\xf0\x03\xa67\x80\xc4=b\x97\x0cY\xb6l\xef\xfa\xa2\xb0\xec\xf5s\x8b\xd3\xd2t\x98dO\x90\x9c\xb1\x15\xb1\x1f\xbf@5\x87D\xa2\x90@\xa2\xaaz\x1c\xbb\x0f\x1e\x8d\x85.\\2Of\x9e<\xf9\x7f\xf2\xdf\xea\xea\xdb\xc7\xd3\xd3\xd3\xd5\xe7\xfd\xdd\xf1\xfd\xfe\xf9\xf4\xf8+0\xef\xae\xbe\x7f<~\xde?\x1f\xde]\x01X/\xe0\xdd\x95\x94\xcf\x1f\xde]\xb9wW\x7f>|><\xee\xee\xf7\x8f\x8f\xc7\xc3\xfbwW\x7f\xda\xdf~8>\x1cv\xa7\x8f\xbb\xe3\xc3\xd3\xc7\xdb\xe7wW\x7f\xf9\xd7\xc3.\xfe\xe1]\xfc\xa7\xbf\xbe\xdb\xdf\xfe3\xfd\xa5\xbb\xf8\xaf\x12\xd3\xff\xe9\xf8\xff\x7f\x7f8>\x1f\xde\xef~x\x8eK<\xbd\xbb\xfa\xcf\xff2\xe2\x7f\xfe\xe3W\xcag\xeb\xfa\xe0\xa5~w\xf5\xfb\x1fv??

In [2]:
!unzip us-census-data.zip

Archive:  us-census-data.zip
  inflating: adult-test.csv          
  inflating: adult-training.csv      


In [0]:
columns = [
'age',
'govworkclass',
'fnlwgt',
'education',
'education_num',
'marital_status',
'clericaloccupation',
'familyrelationship',
'race',
'gender',
'capital_gain',
'capital_loss',
'hours_per_week',
'native_country',
'income_bracket'
]

In [0]:
train = pd.read_csv('adult-training.csv', header=None, names=columns)

In [0]:
test = pd.read_csv('adult-test.csv', names=columns)

In [0]:
test.drop(0, inplace=True)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age                   32561 non-null int64
govworkclass          32561 non-null object
fnlwgt                32561 non-null int64
education             32561 non-null object
education_num         32561 non-null int64
marital_status        32561 non-null object
clericaloccupation    32561 non-null object
familyrelationship    32561 non-null object
race                  32561 non-null object
gender                32561 non-null object
capital_gain          32561 non-null int64
capital_loss          32561 non-null int64
hours_per_week        32561 non-null int64
native_country        32561 non-null object
income_bracket        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [0]:
test.age = test.age.astype(int)
test.fnlwgt = test.fnlwgt.astype(int)
test.education_num = test.education_num.astype(int)
test.capital_gain = test.capital_gain.astype(int)
test.capital_loss = test.capital_loss.astype(int)
test.hours_per_week = test.hours_per_week.astype(int)

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16281 entries, 1 to 16281
Data columns (total 15 columns):
age                   16281 non-null int64
govworkclass          16281 non-null object
fnlwgt                16281 non-null int64
education             16281 non-null object
education_num         16281 non-null int64
marital_status        16281 non-null object
clericaloccupation    16281 non-null object
familyrelationship    16281 non-null object
race                  16281 non-null object
gender                16281 non-null object
capital_gain          16281 non-null int64
capital_loss          16281 non-null int64
hours_per_week        16281 non-null int64
native_country        16281 non-null object
income_bracket        16281 non-null object
dtypes: int64(6), object(9)
memory usage: 2.0+ MB


In [9]:
test.income_bracket.value_counts()

 <=50K.    12435
 >50K.      3846
Name: income_bracket, dtype: int64

In [0]:
train.income_bracket = train.income_bracket.str.strip()
train.income_bracket = (train.income_bracket != '<=50K').astype(int)
train = pd.get_dummies(train)

test.income_bracket = test.income_bracket.str.strip()
test.income_bracket = (test.income_bracket != '<=50K.').astype(int)
test.age = test.age.astype(int)
test = pd.get_dummies(test)

In [0]:
[x for x in train.columns if x not in test.columns]
test['native_country_ Holand-Netherlands'] = np.zeros((test.shape[0]))
test['native_country_ Holand-Netherlands'] = test['native_country_ Holand-Netherlands'].astype(int)

In [0]:
model = LogisticRegression()

In [0]:
X_train = train.drop('income_bracket', axis=1)
X_test = test.drop('income_bracket', axis=1)
y_train = train['income_bracket']
y_test = test['income_bracket']

In [14]:
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
y_preds = model.predict(X_test)

In [16]:
accuracy_score(y_test, y_preds)

0.7991523862170629

In [17]:
f1_score(y_test, y_preds)

0.37903532092669956

In [18]:
roc_auc_score(y_test, y_preds)

0.6127769549865227

In [19]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.81      0.97      0.88     12435
           1       0.70      0.26      0.38      3846

   micro avg       0.80      0.80      0.80     16281
   macro avg       0.76      0.61      0.63     16281
weighted avg       0.78      0.80      0.76     16281

